# SLAMseq

## Installing slamdunk (only do once)

In [1]:
# install slamdunk
#! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [2]:
# clone and install genepy
# pip install -e genepy

In [3]:
# please also install cutadapt, fastqc, and trimgalore
#! conda install -c bioconda cutadapt
#! conda install -c bioconda fastqc
#! conda install -c bioconda trim-galore

In [1]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
import seaborn as sns
import numpy as np
from natsort import os_sorted

from genepy.utils import helper as h
from genepy.utils import plot as genepyPlot
from genepy.rna import pyDESeq2
import genepy.rna as rna

In [3]:
from bokeh.plotting import *
from bokeh.models import HoverTool
from bokeh.io import output_notebook

import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

output_notebook()
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load_ext rpy2.ipython

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
def createDir(directory):
    if not os.path.exists(directory):
        print("Creating output directory: " + directory)
        os.makedirs(directory)

# MYB-dTAG in multiple cell lines

SLAMseq of MYB-dTag in multiple cell lines (MV411, MONOMAC1, NOMO1, P31FUJ, THP1)
<br>All MYB CT degrons with P2A sequence between degron and FP
<br>1h degradation + 1h 4sU with 0.5$\mu$M VHL (with ERCC spike-in control)
<br>__Processed without alt loci__

In [5]:
project = "MYB_degraded_lines_v1"

In [6]:
location = "../data/slamseq_"+project+"/" # new data location

## download the data

In [9]:
location = "../data/slamseq_"+project+"/" # new data location
loc = "220602_MP10090_fastq" # transfer AML folder

In [10]:
! mkdir ../$location
! mkdir ../results/slamseq_$project
! mkdir $location && mkdir $location/qc # make data dir in AMLproject
! mkdir ../$location/fastqs/ # make dir in /home/monika/data/project/fastqs

In [34]:
! gsutil ls gs://transfer-amlproject/$loc/

gs://transfer-amlproject/220602_MP10090_fastq/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R1_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R2_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57_R1_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57_R2_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58_R1_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58_R2_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_13_MYB-dTAG_1h_MONOMAC1_VHL_A_MP10090_S59_R1_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_13_MYB-dTAG_1h_MONOMAC1_VHL_A_MP10090_S59_R2_001.fastq.gz
gs://transfer-amlproject/220602_MP10090_fastq/20220602_14_MYB-dTAG_1h_MONOMAC1_VHL_B_MP100

In [33]:
! gsutil ls gs://transfer-amlproject/*fastq.gz

gs://transfer-amlproject/20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62_R1_001.fastq.gz
gs://transfer-amlproject/20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62_R2_001.fastq.gz


In [32]:
! gsutil -m cp gs://transfer-amlproject/*fastq.gz gs://transfer-amlproject/$loc/

Copying gs://transfer-amlproject/20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62_R2_001.fastq.gz [Content-Type=application/octet-stream]...
/ [2/2 files][  6.9 GiB/  6.9 GiB] 100% Done                                    
Operation completed over 2 objects/6.9 GiB.                                      


In [12]:
# transfer qc files
! gsutil -m cp gs://transfer-amlproject/$loc/multiqc_report.html $location/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/Reports/ $location/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/multiqc_data/ $location/qc/

Copying gs://transfer-amlproject/220602_MP10090_fastq/multiqc_report.html...
/ [1/1 files][  2.0 MiB/  2.0 MiB] 100% Done                                    
Operation completed over 1 objects/2.0 MiB.                                      
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_11_MYB-dTAG

Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_24_MYB-dTAG_1h_NOMO1_VHL_D_MP10090_S70_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_25_MYB-dTAG_1h_P31FUJ_DMSO_A_MP10090_S71_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_25_MYB-dTAG_1h_P31FUJ_DMSO_A_MP10090_S71_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_25_MYB-dTAG_1h_P31FUJ_DMSO_A_MP10090_S71_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_26_MYB-dTAG_1h_P31FUJ_DMSO_B_MP10090_S72_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_26_MYB-dTAG_1h_P31FUJ_DMSO_B_MP10090_S72_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_26_MYB-dTAG_1h_P31FUJ_DMSO_B_MP10090_S72_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/202206

Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_40_MYB-dTAG_1h_THP1_VHL_D_MP10090_S86_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_40_MYB-dTAG_1h_THP1_VHL_D_MP10090_S86_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_40_MYB-dTAG_1h_THP1_VHL_D_MP10090_S86_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220602_MP10090_fastq/Reports/20220602_5_MYB-dTAG_

In [13]:
! cp $location/qc/multiqc_report.html ../results/slamseq_$project

In [35]:
fastqs = ! gsutil ls gs://transfer-amlproject/$loc/*.gz

add sample names to AML sample tracker google sheet (RNA_tracker_JK)

__6_MYB-dTAG_1h_MV411_VHL_B__: failed initial QC (not enough RNA)

In [36]:
print(len(fastqs))
[f.split("_R")[0] for f in fastqs[::2]]

78


['gs://transfer-amlproject/220602_MP10090_fastq/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_13_MYB-dTAG_1h_MONOMAC1_VHL_A_MP10090_S59',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_14_MYB-dTAG_1h_MONOMAC1_VHL_B_MP10090_S60',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_15_MYB-dTAG_1h_MONOMAC1_VHL_C_MP10090_S61',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_17_MYB-dTAG_1h_NOMO1_DMSO_A_MP10090_S63',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_18_MYB-dTAG_1h_NOMO1_DMSO_B_MP10090_S64',
 'gs://transfer-amlproject/220602_MP10090_fastq/20220602_19_MYB-dTAG_1h_NOMO1_DMSO_C_MP10090

In [40]:
fastqs = [fastq.split("/")[-1] for fastq in os_sorted(fastqs)]
# add sample names to AML sample tracker google sheet
for val1, val2 in h.grouped(fastqs, 2):
    print(val1.split("_R")[0])

20220602_1_MYB-dTAG_1h_MV411_DMSO_A_MP10090_S48
20220602_2_MYB-dTAG_1h_MV411_DMSO_B_MP10090_S49
20220602_3_MYB-dTAG_1h_MV411_DMSO_C_MP10090_S50
20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51
20220602_5_MYB-dTAG_1h_MV411_VHL_A_MP10090_S52
20220602_7_MYB-dTAG_1h_MV411_VHL_C_MP10090_S53
20220602_8_MYB-dTAG_1h_MV411_VHL_D_MP10090_S54
20220602_9_MYB-dTAG_1h_MONOMAC1_DMSO_A_MP10090_S55
20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56
20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57
20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58
20220602_13_MYB-dTAG_1h_MONOMAC1_VHL_A_MP10090_S59
20220602_14_MYB-dTAG_1h_MONOMAC1_VHL_B_MP10090_S60
20220602_15_MYB-dTAG_1h_MONOMAC1_VHL_C_MP10090_S61
20220602_16_MYB-dTAG_1h_MONOMAC1_VHL_D_MP10090_S62
20220602_17_MYB-dTAG_1h_NOMO1_DMSO_A_MP10090_S63
20220602_18_MYB-dTAG_1h_NOMO1_DMSO_B_MP10090_S64
20220602_19_MYB-dTAG_1h_NOMO1_DMSO_C_MP10090_S65
20220602_20_MYB-dTAG_1h_NOMO1_DMSO_D_MP10090_S66
20220602_21_MYB-dTAG_1h_NOMO1_VHL_A_MP10090_S67
20220602_22_

In [41]:
# transfer fastqs
! gsutil -m cp gs://transfer-amlproject/$loc/*.fastq.gz ../$location/fastqs/

Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_10_MYB-dTAG_1h_MONOMAC1_DMSO_B_MP10090_S56_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_11_MYB-dTAG_1h_MONOMAC1_DMSO_C_MP10090_S57_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_12_MYB-dTAG_1h_MONOMAC1_DMSO_D_MP10090_S58_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_13_MYB-dTAG_1h_MONOMAC1_VHL_A_MP10090_S59_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220602_MP10090_fastq/20220602_14_MYB-dTAG_1h_MONOMAC1_VHL_B_MP10090_S60_R1_001.fastq.gz...
Co

## rename local fastqs

In [60]:
fastq_folder = "../"+location+"fastqs"
print(fastq_folder)
fastqs = ! ls $fastq_folder
fastqs = os_sorted(fastqs)
fastqs

../../data/slamseq_MYB_degraded_lines_v1/fastqs


['20220602_1_MYB-dTAG_1h_MV411_DMSO_A_MP10090_S48_R1_001.fastq.gz',
 '20220602_1_MYB-dTAG_1h_MV411_DMSO_A_MP10090_S48_R2_001.fastq.gz',
 '20220602_2_MYB-dTAG_1h_MV411_DMSO_B_MP10090_S49_R1_001.fastq.gz',
 '20220602_2_MYB-dTAG_1h_MV411_DMSO_B_MP10090_S49_R2_001.fastq.gz',
 '20220602_3_MYB-dTAG_1h_MV411_DMSO_C_MP10090_S50_R1_001.fastq.gz',
 '20220602_3_MYB-dTAG_1h_MV411_DMSO_C_MP10090_S50_R2_001.fastq.gz',
 '20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R1_001.fastq.gz',
 '20220602_4_MYB-dTAG_1h_MV411_DMSO_D_MP10090_S51_R2_001.fastq.gz',
 '20220602_5_MYB-dTAG_1h_MV411_VHL_A_MP10090_S52_R1_001.fastq.gz',
 '20220602_5_MYB-dTAG_1h_MV411_VHL_A_MP10090_S52_R2_001.fastq.gz',
 '20220602_7_MYB-dTAG_1h_MV411_VHL_C_MP10090_S53_R1_001.fastq.gz',
 '20220602_7_MYB-dTAG_1h_MV411_VHL_C_MP10090_S53_R2_001.fastq.gz',
 '20220602_8_MYB-dTAG_1h_MV411_VHL_D_MP10090_S54_R1_001.fastq.gz',
 '20220602_8_MYB-dTAG_1h_MV411_VHL_D_MP10090_S54_R2_001.fastq.gz',
 '20220602_9_MYB-dTAG_1h_MONOMAC1_DMSO_A_MP10090_S55_R

In [47]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[12].to_frame()

In [61]:
len(fastqs)/2

39.0

In [70]:
# get new sample names from AML sample tracker - RNA_tracker_JK
idx = gsheet.index[gsheet['batch'] == project] # get first row with project number
sample_info = gsheet.loc[idx[0]:, ["prev_name", "name"]]
sample_info = sample_info.iloc[:int(len(fastqs)/2),]

In [71]:
# make dict with names
rename = dict(zip(sample_info.prev_name, sample_info.name))

# rename fastqs locally
for val in fastqs:
    rep  = val
    for old, new in rename.items():
        rep = rep.replace(old, new)
    if rep != val:
        ! mv $fastq_folder/$val $fastq_folder/$rep

# Processing

In [7]:
project

'MYB_degraded_lines_v1'

In [21]:
fastq_folder = "../"+location+"fastqs"
print(fastq_folder)
fastqs = ! ls $fastq_folder/*fastq.gz
print(len(fastqs[::2]))
fastqs[::2]

../../data/slamseq_MYB_degraded_lines_v1/fastqs
39


['../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr532-MV411-MYB-dTAG-DMSO_1h-r1_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr533-MV411-MYB-dTAG-DMSO_1h-r2_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr534-MV411-MYB-dTAG-DMSO_1h-r3_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr535-MV411-MYB-dTAG-DMSO_1h-r4_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr536-MV411-MYB-dTAG-VHL_1h-r1_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr537-MV411-MYB-dTAG-VHL_1h-r2_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr538-MV411-MYB-dTAG-VHL_1h-r3_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr539-MONOMAC1-MYB-dTAG-DMSO_1h-r1_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr540-MONOMAC1-MYB-dTAG-DMSO_1h-r2_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr541-MONOMAC1-MYB-dTAG-DMSO_1h-r3_R1_001.fastq

## trim_galore

In [24]:
for val1, val2 in h.grouped(fastqs[58:60]+fastqs[62:64], 2):
    print(os.path.basename(val1))

mr561-P31FUJ-MYB-dTAG-VHL_1h-r3_R1_001.fastq.gz
mr563-THP1-MYB-dTAG-DMSO_1h-r1_R1_001.fastq.gz


In [10]:
for val1, val2 in h.grouped(fastqs[68:], 2):
    print(os.path.basename(val1))

mr566-THP1-MYB-dTAG-DMSO_1h-r4_R1_001.fastq.gz
mr567-THP1-MYB-dTAG-VHL_1h-r1_R1_001.fastq.gz
mr568-THP1-MYB-dTAG-VHL_1h-r2_R1_001.fastq.gz
mr569-THP1-MYB-dTAG-VHL_1h-r3_R1_001.fastq.gz
mr570-THP1-MYB-dTAG-VHL_1h-r4_R1_001.fastq.gz


In [83]:
%%time
h.parrun(['trim_galore --paired --cores 6 --retain_unpaired -stringency 3\
 --illumina '+val1+' '+val2+' -o '+fastq_folder for val1, val2 in h.grouped(fastqs[56:], 2)], 2)

ValueError: issue with the command: b'Path to Cutadapt set as: \'cutadapt\' (default)\nPath to Cutadapt set as: \'cutadapt\' (default)\nCutadapt seems to be working fine (tested command \'cutadapt --version\')\nCutadapt seems to be working fine (tested command \'cutadapt --version\')\nCutadapt version: 3.4\nCutadapt version: 3.4\nCould not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)\nLetting the (modified) Cutadapt deal with the Python version instead\nCould not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)\nLetting the (modified) Cutadapt deal with the Python version instead\nParallel gzip (pigz) detected. Proceeding with multicore (de)compression using 6 cores\n\nNo quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)\n\nOutput will be written into the directory: /home/monika/data/slamseq_MYB_degraded_lines

In [11]:
%%time
h.parrun(['trim_galore --paired --cores 6 --retain_unpaired -stringency 3\
 --illumina '+val1+' '+val2+' -o '+fastq_folder for val1, val2 in h.grouped(fastqs[68:], 2)], 2)

CPU times: user 236 ms, sys: 48 ms, total: 284 ms
Wall time: 2h 13min 5s


['pigz 2.6\npigz 2.6\n', 'pigz 2.6\npigz 2.6\n', 'pigz 2.6\n']

In [25]:
%%time
h.parrun(['trim_galore --paired --cores 6 --retain_unpaired -stringency 3\
 --illumina '+val1+' '+val2+' -o '+fastq_folder for val1, val2 in h.grouped(fastqs[58:60]+fastqs[62:64], 2)], 2)

CPU times: user 80 ms, sys: 24 ms, total: 104 ms
Wall time: 49min 54s


['pigz 2.6\npigz 2.6\n']

## slamdunk

install slamdunk & dependencies if needed (see above)

In [ ]:
# install slamdunk
#! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [ ]:
# please also install cutadapt, fastqc, and trimgalore
#! conda install -c bioconda cutadapt
#! conda install -c bioconda fastqc
#! conda install -c bioconda trim-galore

download reference genome files

In [ ]:
# google storage bucket with reference files
#! gsutil ls gs://ccle_default_params

# copy reference genome files
#! gsutil -m cp gs://ccle_default_params/Homo_sapiens_assembly38* ../../ref/

# copy reference genome files
#! cp ../data/Muhar_Slamseq/{ERCC92.fa,ERCC92.gtf,GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed} ../../ref

In [12]:
# using an ERCC ref genome (you can just append ERCC fasta to the hg38 fasta)
refgenome_noAlt="../../ref/Homo_sapiens_assembly38_ERCC92_noAlt.fasta"

In [13]:
# gene intervals bed (use for all projects)
gene_intervals = "../../ref/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed"

PySam warning can be ignored.
<br>[GitHub issue: [E::idx_find_and_load] Could not retrieve index file" when AlignmentFile](https://github.com/pysam-developers/pysam/issues/939)

res = slamdunk all w/ (-fb, -m, -n 1, -rl 152, -r Homo_sapiens_assembly38_ERCC92_noAlt.fasta)

Read length note!
<br>Difference between min and max read length is > 100bp
<br>Average read length 127 (min: 8, max: 152)

In [26]:
fastq_folder = "../../data/"+"slamseq_"+project+"/fastqs"
fastqs = ! ls $fastq_folder
fastqs = [fastq for fastq in fastqs if "val" in fastq]
fastq_full = [fastq_folder+"/"+f for f in fastqs]
len(fastq_full[::2])

39

In [27]:
res_dir = "../../data/"+"slamseq_"+project+"/res"
res_dir

'../../data/slamseq_MYB_degraded_lines_v1/res'

In [28]:
%%time

# print call
print(['slamdunk all'+
       ' -o '+res_dir+
       ' -r '+refgenome_noAlt+
       ' -b '+gene_intervals+
       ' -t 14'+
       ' -c 2'+
       ' -rl 152'+ # max-read-length = 152
       # new parameters
       ' -5 12'+
       ' -n 1'+ # reads map to multiple locations at once
       ' -m'+ # multimapping
       ' -N '+val1.split('/')[-1].split('_R')[0]+
       ' '+val1+" "+val2
       for val1, val2 in h.grouped(fastq_full[:2], 2)])

['slamdunk all -o ../../data/slamseq_MYB_degraded_lines_v1/res -r ../../ref/Homo_sapiens_assembly38_ERCC92_noAlt.fasta -b ../../ref/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed -t 14 -c 2 -rl 152 -5 12 -n 1 -m -N mr532-MV411-MYB-dTAG-DMSO_1h-r1 ../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr532-MV411-MYB-dTAG-DMSO_1h-r1_R1_001_val_1.fq.gz ../../data/slamseq_MYB_degraded_lines_v1/fastqs/mr532-MV411-MYB-dTAG-DMSO_1h-r1_R2_001_val_2.fq.gz']
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 168 µs


In [29]:
for val1, val2 in h.grouped(fastq_full, 2): print(os.path.basename(val1))

mr532-MV411-MYB-dTAG-DMSO_1h-r1_R1_001_val_1.fq.gz
mr533-MV411-MYB-dTAG-DMSO_1h-r2_R1_001_val_1.fq.gz
mr534-MV411-MYB-dTAG-DMSO_1h-r3_R1_001_val_1.fq.gz
mr535-MV411-MYB-dTAG-DMSO_1h-r4_R1_001_val_1.fq.gz
mr536-MV411-MYB-dTAG-VHL_1h-r1_R1_001_val_1.fq.gz
mr537-MV411-MYB-dTAG-VHL_1h-r2_R1_001_val_1.fq.gz
mr538-MV411-MYB-dTAG-VHL_1h-r3_R1_001_val_1.fq.gz
mr539-MONOMAC1-MYB-dTAG-DMSO_1h-r1_R1_001_val_1.fq.gz
mr540-MONOMAC1-MYB-dTAG-DMSO_1h-r2_R1_001_val_1.fq.gz
mr541-MONOMAC1-MYB-dTAG-DMSO_1h-r3_R1_001_val_1.fq.gz
mr542-MONOMAC1-MYB-dTAG-DMSO_1h-r4_R1_001_val_1.fq.gz
mr543-MONOMAC1-MYB-dTAG-VHL_1h-r1_R1_001_val_1.fq.gz
mr544-MONOMAC1-MYB-dTAG-VHL_1h-r2_R1_001_val_1.fq.gz
mr545-MONOMAC1-MYB-dTAG-VHL_1h-r3_R1_001_val_1.fq.gz
mr546-MONOMAC1-MYB-dTAG-VHL_1h-r4_R1_001_val_1.fq.gz
mr547-NOMO1-MYB-dTAG-DMSO_1h-r1_R1_001_val_1.fq.gz
mr548-NOMO1-MYB-dTAG-DMSO_1h-r2_R1_001_val_1.fq.gz
mr549-NOMO1-MYB-dTAG-DMSO_1h-r3_R1_001_val_1.fq.gz
mr550-NOMO1-MYB-dTAG-DMSO_1h-r4_R1_001_val_1.fq.gz
mr551-NOMO1-MY

In [36]:
for val1, val2 in h.grouped(fastq_full[6:10], 2): print(os.path.basename(val1))

mr535-MV411-MYB-dTAG-DMSO_1h-r4_R1_001_val_1.fq.gz
mr536-MV411-MYB-dTAG-VHL_1h-r1_R1_001_val_1.fq.gz


In [ ]:
%%time

# run slamdunk
# we are doing it paired end
h.parrun(['slamdunk all'+
          ' -o '+res_dir+
          ' -r '+refgenome_noAlt+
          ' -b '+gene_intervals+
          ' -t 14'+
          ' -c 2'+
          ' -rl 152'+ # max-read-length = 152
          # new parameters
          ' -5 12'+
          ' -n 1'+ # reads map to multiple locations at once
          ' -m True'+ # multimapping
          ' -N '+val1.split('/')[-1].split('_R')[0]+
          ' '+val1+" "+val2+
          ' 2>&1 | tee '+res_dir+'/'+os.path.basename(val1)+'_slumdunk.log'
          for val1, val2 in h.grouped(fastq_full[6:10], 2)], 2)